## Dependency Parsing as a Preprocessing Step for Logical Reasoning

In [1]:
# Installs HBOX for Jupyer Notebooks
# !jupyter labextension install @jupyter-widgets/jupyterlab-manager
# !jupyter nbextension enable --py widgetsnbextension
# drive_path = ''

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
drive_path = '/content/drive/MyDrive/FinalProject/'
!pip install transformers

Mounted at /content/drive
     |████████████████████████████████| 3.4 MB 12.6 MB/s 
     |████████████████████████████████| 3.3 MB 56.2 MB/s 
     |████████████████████████████████| 596 kB 49.5 MB/s 
     |████████████████████████████████| 895 kB 50.6 MB/s 
     |████████████████████████████████| 61 kB 495 kB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import os
import gc
import re
import json
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
from transformers import LongformerTokenizer
from torch.utils.data import TensorDataset, Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import LongformerConfig, LongformerForSequenceClassification, AdamW

# Enable CUDA Blocking Debugging
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
CUDA_LAUNCH_BLOCKING="1"

# Print GPU Information
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
    print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
    print('and then re-execute this cell.')
else:
    print(gpu_info)

Wed Dec 22 04:58:51 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

#### Load Data

In [3]:
def load_data(path, test):
    df = pd.DataFrame({'prompt': [], 'label': []})
    data = json.load(open(path))
    for i in range(len(data['context'])):
        # Add BERT tokens to prompt
        prompt = data['context'][str(i)] + ' </s> ' + data['question'][str(i)] + ' </s> '# + ' <d> ' + data['dep_context'][str(i)] + ' </s> '
        for j in range(4):
            # Add BERT tokens to answer
            answer = data['answers'][str(i)][j]# + ' <d> ' + data['dep_answers'][str(i)][j]
            
            # Attach 0, 1 label as array
            label = [1] if not test and j == data['label'][str(i)] else [0]
            
            # Append question, answer pair to dataframe
            df = df.append({'prompt': prompt + answer, 'label': [label]}, ignore_index=True)            
    return df

In [4]:
train_data = load_data(drive_path + 'reclor_data_with_dependencies/train.json', False)
val_data = load_data(drive_path + 'reclor_data_with_dependencies/val.json', False)
test_data = load_data(drive_path + 'reclor_data_with_dependencies/test.json', True)

In [5]:
#val_data['prompt'].apply(lambda x: len(re.findall(r'([^\s]+)', x))).describe()

In [6]:
# Describe the token lengths of training data
#train_data['prompt'].apply(lambda x: len(re.findall(r'([^\s]+)', x))).describe()



#### Initialize tokenizer

In [7]:
tokenizer = LongformerTokenizer.from_pretrained('allenai/longformer-base-4096')
tokenizer.add_special_tokens({'additional_special_tokens': ['<d>']})

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/694 [00:00<?, ?B/s]

1

#### Tokenize the data

In [8]:
# Creates a dataloader (which includes an attention mask)
def preprocess(in_, tokenizer, max_len, batch_size, data_class='train'):
    encoded_input = tokenizer(in_['prompt'].values.tolist(), padding=True, max_length=max_len, truncation=True, return_tensors="pt")
    
    if data_class != 'test':
        labels = torch.tensor(in_['label'].values.tolist())

    dataset_tensor = TensorDataset(encoded_input['input_ids'], encoded_input['attention_mask'], labels)
    
    sampler = SequentialSampler(dataset_tensor) #RandomSampler(dataset_tensor) if data_class == "train" else SequentialSampler(dataset_tensor)
    dataloader = DataLoader(dataset_tensor, sampler=sampler, batch_size=batch_size)
    return dataloader

In [9]:
max_len = 1024 # should be 1024
batch_size = 4

train_dataloader = preprocess(train_data, tokenizer, max_len, batch_size)
val_dataloader = preprocess(val_data, tokenizer, max_len, batch_size, data_class="val")
# test_dataloader = preprocess(test_data, tokenizer, max_len, batch_size, data_class="test")

#### Train and Evaluate RoBERTa

In [32]:
def ClearTorch():
    torch.no_grad()
    torch.cuda.empty_cache()
    gc.collect()
    
def Eval(model, dataloader):
    ClearTorch()
    model.eval()
    predictions, true_labels = [], []
    for step, batch in enumerate(tqdm(dataloader)):
        # Call model on batch
        input_ids, attention_mask, labels = batch[0].cuda(), batch[1].cuda(), batch[2].cuda()
        temp = np.array([1])
        global_attention_mask = np.append(temp, [0] * (len(input_ids[0]) - 1))
        gam = torch.tensor([global_attention_mask])
        global_attention_mask = torch.stack( (gam, gam, gam, gam), dim=1).cuda()

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        
        # Convert output logit to predictions using softmax
        ClearTorch()

        predictions.append(torch.nn.functional.softmax(outputs.logits).argmax(0)[1].cpu().numpy().tolist())
        true_labels.append(labels.argmax(0).cpu().numpy().tolist()[0][0])
      
        ClearTorch()

    return float(sum([predictions[i] == true_labels[i] for i in range(len(predictions)) ])) / float(len(predictions))

def Train(model, train_data, lr, n_epoch):
    ClearTorch()
    optimizer = AdamW(model.parameters(), lr=lr)

    for epoch in range(n_epoch):
        print(f"Epoch {epoch}")
        model.train()
        nb_tr_examples, nb_tr_steps, tr_loss = 0, 0, 0

        for step, batch in enumerate(tqdm(train_data)):
            # RoBERTa fine-tuning

            input_ids, attention_mask, labels = batch[0].cuda(), batch[1].cuda(), batch[2].cuda()
            temp = np.array([1])
            global_attention_mask = np.append(temp, [0] * (len(input_ids[0]) - 1))
            gam = torch.tensor([global_attention_mask])
            global_attention_mask = torch.stack( (gam, gam, gam, gam), dim=1).cuda()

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels, global_attention_mask = global_attention_mask)
            outputs.loss.backward()
            optimizer.step()
            
            ClearTorch()
            
            tr_loss += float(outputs.loss)
            nb_tr_steps += 1
            
        print(f"Train loss on epoch {epoch}: {tr_loss / nb_tr_steps}\n")

In [33]:
ClearTorch()

config = LongformerConfig.from_pretrained('allenai/longformer-base-4096')
# config.max_position_embeddings = max_len
model = LongformerForSequenceClassification.from_pretrained('allenai/longformer-base-4096', config=config)
model.resize_token_embeddings(len(tokenizer))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if torch.cuda.device_count() > 1:
    model.to(device)
    model = torch.nn.DataParallel(model)
else:
    model.cuda()
    
learning_rate = 2e-5
num_epoch = 1

Train(model, train_dataloader, learning_rate, num_epoch)
Eval(model, val_dataloader)

Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerForSequenceClassification: ['lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing LongformerForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weig

Epoch 0


100%|██████████| 4638/4638 [1:26:52<00:00,  1.12s/it]


Train loss on epoch 0: 0.5647187890190151



100%|██████████| 500/500 [05:29<00:00,  1.52it/s]


0.272

In [ ]:
def CPUEval(model, dataloader):
    ClearTorch()
    model.eval()
    predictions, true_labels = [], []
    for step, batch in enumerate(tqdm(dataloader)):
        # Call model on batch
        input_ids, attention_mask, labels = batch[0], batch[1], batch[2]
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        
        # Convert output logit to predictions using softmax
        ClearTorch()

        predictions.append(torch.nn.functional.softmax(outputs.logits).argmax(0)[1].cpu().numpy().tolist())
        true_labels.append(labels.argmax(0).cpu().numpy().tolist()[0][0])
      
        ClearTorch()

    return float(sum([predictions[i] == true_labels[i] for i in range(len(predictions)) ])) / float(len(predictions))

In [ ]:
torch.save(model, '/content/drive/MyDrive/model.pth')

In [ ]:
model.to('cpu')
CPUEval(model, val_dataloader)